In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import py_trees
import logging
from deep_traffic.libtraffic import env
from behavior_tree.agent import BasicBehaviorTreeAgent, GeneticProgrammingBehaviorTreeAgent
from behavior_tree.condition_checker import BasicConditionChecker, GeneticProgrammingConditionChecker
from behavior_tree.tree import BasicBehaviorTree, GeneticProgrammingBehaviorTree, ConditionSequenceNode, IntersectConditionSequenceNode
from train import train_GP_BT, train_GP_BT_v2
from runner import play_episode, measure_performance
from config import *

In [ ]:
logging.basicConfig(level=logging.INFO)

##### Environment

In [ ]:
e = env.DeepTraffic(lanes_side=LANES_SIDE, 
                    patches_ahead=PATCHES_AHEAD,
                    patches_behind=PATCHES_BEHIND, 
                    history=HISTORY, 
                    obs=OBS)
obs_shape = e.obs_shape
print(obs_shape)

In [ ]:
state = e.reset()
print(state.shape)

In [ ]:
print(state[0].T)

##### ConditionChecker

In [ ]:
condition_checker = GeneticProgrammingConditionChecker()
condition_checker.update_state(state[0])

In [ ]:
print(condition_checker.state.shape)

In [ ]:
print(condition_checker.state.T)

##### BTAgent

In [ ]:
agent = GeneticProgrammingBehaviorTreeAgent()

In [ ]:
# load agent
agent.load(name="GP-BT.dat")
agent.setup()

In [ ]:
# display tree
agent.display_tree()

In [ ]:
# tick
agent.update_condition_checker(state[0])
agent.update_blackboard()
print(agent.blackboard)

##### Training

In [ ]:
%%time
#py_trees.logging.level = py_trees.logging.Level.DEBUG
train_GP_BT(e, agent, episodes=TRAIN_EPISODES, steps=PLAY_EPISODE_STEPS)

In [ ]:
# display tree
agent.display_tree()

##### Play Episode

In [ ]:
#py_trees.logging.level = py_trees.logging.Level.DEBUG
agent.blackboard.enable_learning = False
play_episode(e, agent, verbose=True, reset=True)

##### Multiple Episodes

In [ ]:
%%time
measure_performance(e, agent, episodes=100)